In [1]:
import plotly.express as px
import pandas as pd
import sqlite3

In [2]:
# Create a connection to the database
conn = sqlite3.connect("C:/Users/alexp/Documents/GTM/db.sqlite3")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute SQL queries to extract tables
cursor.execute("SELECT * FROM workout_workout")
workout_data = cursor.fetchall()

cursor.execute("SELECT * FROM workout_exercise")
exercise_data = cursor.fetchall()

cursor.execute("SELECT * FROM workout_category")
category_data = cursor.fetchall()

# Convert the data to a pandas DataFrame
workout_df = pd.DataFrame(workout_data, columns=["id", "sets", "reps", "weight", "exercise_id", "date"])
exercise_df = pd.DataFrame(exercise_data, columns=["id", "name", "category_id"])
category_df = pd.DataFrame(category_data, columns=["id", "category"])

# Close the cursor and connection
cursor.close()
conn.close()

In [3]:
workout_df.drop(columns=["id"], inplace=True)

In [4]:
merged_df1 = workout_df.merge(exercise_df, left_on='exercise_id', right_on='id')

In [5]:
merged_df = merged_df1.merge(category_df, left_on='category_id', right_on='id')

In [6]:
merged_df.sort_values(by='date', inplace=True)
merged_df.drop(columns=["exercise_id", "category_id", "id_x", "id_y"], inplace=True)
merged_df.rename(columns={'name': 'exercise'}, inplace=True)

In [8]:
fig = px.bar(merged_df['exercise'].value_counts(), x=merged_df['exercise'].value_counts().index, y=merged_df['exercise'].value_counts().values, template="plotly_dark")
fig.update_layout(title="# of workouts over the last year", xaxis_title="Exercise Name", yaxis_title="# of times completed", xaxis_tickangle=-45)
fig.add_hline(y=merged_df['exercise'].value_counts().mean(), line_dash="dot", line_color="red", annotation_text="Average", annotation_position="bottom right")
fig.show()

In [9]:
lower_body_df = merged_df[merged_df['category'] == 'Lower Body']
fig = px.bar(lower_body_df['exercise'].value_counts(), x=lower_body_df['exercise'].value_counts().index, y=lower_body_df['exercise'].value_counts().values, template="plotly_dark")
fig.update_layout(title="# of Lower Body workouts over the last year", xaxis_title="Exercise Name", yaxis_title="# of times completed", xaxis_tickangle=-45)
fig.add_hline(y=lower_body_df['exercise'].value_counts().mean(), line_dash="dot", line_color="red", annotation_text="Average", annotation_position="bottom right")
fig.show()


In [10]:
upper_body_df = merged_df[merged_df['category'] == 'Upper Body']
fig = px.bar(upper_body_df['exercise'].value_counts(), x=upper_body_df['exercise'].value_counts().index, y=upper_body_df['exercise'].value_counts().values, template="plotly_dark")
fig.update_layout(title="# of Upper Body workouts over the last year", xaxis_title="Exercise Name", yaxis_title="# of times completed", xaxis_tickangle=-45)
fig.add_hline(y=upper_body_df['exercise'].value_counts().mean(), line_dash="dot", line_color="red", annotation_text="Average", annotation_position="bottom right")
fig.show()


In [11]:
lower_body_df = merged_df[merged_df['category'] == 'Lower Body']
lower_body_weight_avg = lower_body_df.groupby('exercise')['weight'].mean().reset_index()
lower_body_weight_avg = lower_body_weight_avg[lower_body_weight_avg['weight'] > 0]
lower_body_weight_avg = lower_body_weight_avg.sort_values(by="weight", ascending=False)

fig = px.bar(lower_body_weight_avg, x='exercise', y='weight', template="plotly_dark")
fig.update_layout(title="Average Weight for Lower Body workouts over the last year", xaxis_title="Exercise Name", yaxis_title="Average Weight", xaxis_tickangle=-45)
fig.add_hline(y=lower_body_weight_avg['weight'].mean(), line_dash="dot", line_color="red", annotation_text="Average", annotation_position="bottom right")
fig.show()

In [12]:
max_weight_df = merged_df.groupby('exercise')['weight'].max().reset_index()
max_weight_df.sort_values(by='weight', ascending=False, inplace=True)
max_weight_df = max_weight_df.query("weight > 0 & exercise != 'Box Jumps'")

fig = px.bar(max_weight_df, x='exercise', y='weight', template="plotly_dark")
fig.update_layout(title="Max Weight for Each Exercise", xaxis_title="Exercise Name", yaxis_title="Max Weight")
fig.show()

In [7]:
import plotly.graph_objects as go

# Create a list of unique exercises
exercises = merged_df['exercise'].unique()

# Create a figure
fig = go.Figure()

# Add traces for each exercise
for exercise in exercises:
    exercise_df = merged_df[merged_df['exercise'] == exercise]
    fig.add_trace(go.Bar(
        x=exercise_df['date'],
        y=exercise_df['weight'],
        name=exercise
    ))

# Update layout with dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"visible": [exercise == ex for ex in exercises]}],
                    label=exercise,
                    method="update"
                ) for exercise in exercises
            ]),
            direction="down",
            showactive=True,
        )
    ],
    title="Weight for Each Exercise Over Time",
    xaxis_title="Date",
    yaxis_title="Weight",
    template="plotly_dark"
)

fig.show()

In [10]:
from datetime import datetime, timedelta

import plotly.graph_objects as go

# Calculate the date 10 weeks ago from today
ten_weeks_ago = datetime.now() - timedelta(weeks=10)

# Filter the dataframe to include only the last 10 weeks
filtered_df = merged_df[pd.to_datetime(merged_df['date']) >= ten_weeks_ago]

# Create a list of unique exercises
# exercises = filtered_df["exercise"].unique()

# Create a list of a subset of the exercises
exercises = ["Standing Rows", "Dumbbell Angled Bench Press", "Leg Press", "Calf Press", "Box Jumps", "Single Leg Press", "Shoulder Press", "Goblet Squat", "Kettle Bell Swing", "Seated Rows", "Bicep Curls", "Lat Pull Downs"]

# Create a figure
fig = go.Figure()

# Add traces for each exercise
for exercise in exercises:
    exercise_df = filtered_df[filtered_df["exercise"] == exercise]
    fig.add_trace(go.Bar(x=exercise_df["date"], y=exercise_df["weight"], name=exercise))

# Update layout with dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list(
                [
                    dict(
                        args=[{"visible": [exercise == ex for ex in exercises]}],
                        label=exercise,
                        method="update",
                    )
                    for exercise in exercises
                ]
            ),
            direction="down",
            showactive=True,
        )
    ],
    title="Weight for Each Exercise Over Time (Last 10 Weeks)",
    xaxis_title="Date",
    yaxis_title="Weight",
    template="plotly_white",
)

fig.show()